Install dependencies

In [ ]:
pip install transformers datasets evaluate torch


In [ ]:
pip install -U datasets

In [ ]:
!pip install --upgrade transformers


In [ ]:
pip install accelerate -U

Load and tokenize the dataset

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
dataset["train"] = dataset["train"].select(range(1000))
dataset["validation"] = dataset["validation"].select(range(200)) #Due to limited GPU memory availability, the training and evaluation datasets were reduced to 1000 and 200 samples respectively.
# This allowed the model to train and evaluate successfully without running into out-of-memory issues. The code remains scalable to larger datasets when more compute is available.
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length")

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

Prepare the data collator and model

In [ ]:
from transformers import DataCollatorForLanguageModeling, GPT2LMHeadModel

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)
model = GPT2LMHeadModel.from_pretrained("gpt2")


Define evaluation metrics

In [ ]:
import numpy as np
import torch
from torch.nn import CrossEntropyLoss
import math

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    shift_logits = torch.tensor(logits[..., :-1, :])
    shift_labels = torch.tensor(labels[..., 1:])

    loss_fct = CrossEntropyLoss(ignore_index=-100)
    loss = loss_fct(
        shift_logits.reshape(-1, shift_logits.size(-1)),
        shift_labels.reshape(-1)
    )
    perplexity = math.exp(loss.item())

    top_k = 5
    topk_pred = shift_logits.topk(top_k, dim=-1).indices
    match = (topk_pred == shift_labels.unsqueeze(-1)).any(-1)
    topk_acc = match.float().mean().item()

    return {"perplexity": perplexity, "top_5_accuracy": topk_acc}


Train and evaluate with Trainer

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./gpt2-nextword",
    eval_strategy="epoch",  # Changed from evaluation_strategy
    save_strategy="epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    logging_dir="./logs",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()
metrics = trainer.evaluate()
print(metrics)

Use the model for prediction

In [ ]:
text = "Deep learning enables"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=1, do_sample=True)
next_word = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(next_word)
